# Experiment 9.10 - Causal Behavioral Cloning

In [1]:
import numpy as np
import math
import pandas as pd
from gymnasium import spaces

from causal_gym import Graph, SCM, PCH
from imitation.imitate import *

In [2]:
seed = 0

### Graph Definitions

In [3]:
# from Table 9.13
nodes = [{'name': n} for n in ['Z0', 'X0', 'X1', 'Y']]

g1_edges = [
    {'from_': 'Z0', 'to_': 'X0', 'type_': 'bidirected'},
    {'from_': 'Z0', 'to_': 'X1', 'type_': 'bidirected'},
    {'from_': 'Z0', 'to_': 'Y', 'type_': 'bidirected'},
    {'from_': 'X0', 'to_': 'X1', 'type_': 'directed'},
    {'from_': 'X1', 'to_': 'Y', 'type_': 'directed'}
]
g1_ordering = ['Z0', 'X0', 'X1', 'Y']
G1 = Graph(nodes=nodes, edges=g1_edges)

g2_edges = [
    {'from_': 'Z0', 'to_': 'X0', 'type_': 'bidirected'},
    {'from_': 'Z0', 'to_': 'Y', 'type_': 'directed'},
    {'from_': 'X0', 'to_': 'X1', 'type_': 'directed'},
    {'from_': 'X1', 'to_': 'Y', 'type_': 'directed'}
]
g2_ordering = ['Z0', 'X0', 'X1', 'Y']
G2 = Graph(nodes=nodes, edges=g2_edges)

g3_edges = [
    {'from_': 'Z0', 'to_': 'X0', 'type_': 'bidirected'},
    {'from_': 'Z0', 'to_': 'Y', 'type_': 'directed'},
    {'from_': 'X0', 'to_': 'X1', 'type_': 'directed'},
    {'from_': 'X1', 'to_': 'Y', 'type_': 'directed'}
]
g3_ordering = ['X0', 'Z0', 'X1', 'Y']
G3 = Graph(nodes=nodes, edges=g3_edges)

g4_edges = [
    {'from_': 'Z0', 'to_': 'X0', 'type_': 'bidirected'},
    {'from_': 'Z0', 'to_': 'Y', 'type_': 'bidirected'},
    {'from_': 'Z0', 'to_': 'X1', 'type_': 'directed'},
    {'from_': 'X0', 'to_': 'Y', 'type_': 'directed'},
    {'from_': 'X1', 'to_': 'Y', 'type_': 'directed'}
]
g4_ordering = ['X0', 'Z0', 'X1', 'Y']
G4 = Graph(nodes=nodes, edges=g4_edges)

### SCM/PCH Definitions

In [4]:
def _sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

In [5]:
class RandomBernoulliNode:
    def __init__(self, name, parents, conf_inputs, rng):
        self.name = name
        self.parents = list(parents)
        self.conf_inputs = list(conf_inputs)
        self.rng = rng

        self.w0 = float(self.rng.normal(loc=0.0, scale=1.0))
        self.w_par = {p: float(self.rng.normal(loc=0.0, scale=1.0)) for p in self.parents}
        self.w_conf = {u: float(self.rng.normal(loc=0.0, scale=1.0)) for u in self.conf_inputs}

    def sample(self, values):
        z = self.w0

        for p in self.parents:
            z += self.w_par[p] * float(values[p][-1])

        for u in self.conf_inputs:
            z += self.w_conf[u] * float(values[u])

        p = float(_sigmoid(np.array([z]))[0])
        return 1 if self.rng.random() < p else 0

In [6]:
class G1SCM(SCM):
    def __init__(self, graph, seed=None):
        super().__init__()
        self.rng = np.random.default_rng(seed)

        self.graph = graph
        self.node_names = [n['name'] for n in self.graph.nodes]
        self.index = {name: i for i, name in enumerate(self.node_names)}

        n = len(self.node_names)
        self._dir_adj = np.zeros((n, n), dtype=int)
        self._conf_adj = np.zeros((n, n), dtype=int)

        for e in self.graph.edges:
            i = self.index[e['from_']]
            j = self.index[e['to_']]

            if e['type_'] == 'directed':
                self._dir_adj[i, j] = 1
            elif e['type_'] == 'bidirected':
                self._conf_adj[i, j] = 1
                self._conf_adj[j, i] = 1

        self._conf_pairs = []
        seen = set()
        for a in self.node_names:
            for b in self.node_names:
                if a == b:
                    continue

                if self._conf_adj[self.index[a], self.index[b]] == 1:
                    key = tuple(sorted((a, b)))
                    if key not in seen:
                        seen.add(key)
                        self._conf_pairs.append(key)

        # stochastic bias used to generate many random instances
        self.U_bias = {pair: float(self.rng.uniform(-1.0, 1.0)) for pair in self._conf_pairs}

        # Z <--> X0, Z <--> X1, Z <--> Y
        z_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if 'Z0' in (a, b)]
        self.node_z = RandomBernoulliNode('Z0', parents=[], conf_inputs=z_conf, rng=self.rng)

        # Z <--> X0
        X0_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if set((a, b)) == set(('Z0', 'X0'))]
        self.node_X0 = RandomBernoulliNode('X0', parents=[], conf_inputs=X0_conf, rng=self.rng)

        # Z <--> X1, X0 -> X1
        X1_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if set((a, b)) == set(('Z0', 'X1'))]
        self.node_X1 = RandomBernoulliNode('X1', parents=['X0'], conf_inputs=X1_conf, rng=self.rng)

        # Z <--> Y, X1 -> Y
        y_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if set((a, b)) == set(('Z0', 'Y'))]
        self.node_y = RandomBernoulliNode('Y', parents=['X1'], conf_inputs=y_conf, rng=self.rng)

        self.action_space = spaces.Discrete(2) # binary actions at each step
        self.observation_space = spaces.Dict({
            'Z': spaces.Sequence(spaces.Discrete(2)),
            'X': spaces.Sequence(spaces.Discrete(2))
        })

        self._t = 0
        self._values = {}
        self._U = {}

    def _sample_confounders(self):
        U = {}
        for a, b in self._conf_pairs:
            name = f'U_{a}_{b}'
            p = 1 / (1 + math.exp(-self.U_bias[(a, b)]))
            U[name] = 1 if self.rng.random() < p else 0

        return U

    def _obs(self):
        if self._t == 0:
            return {'Z': self._values['Z0'], 'X': []}
        elif self._t == 1:
            return {'Z': self._values['Z0'], 'X': [self._values['X0'][0]]}

        return {'Z': self._values['Z0'], 'X': [self._values['X0'][0], self._values['X1'][0]]}

    def reset(self, seed=None, options=None):
        if seed is not None:
            self.rng = np.random.default_rng(seed)

        self._values = {}
        self._U = self._sample_confounders()

        vals = {u: v for u, v in self._U.items()}
        z = self.node_z.sample(vals)
        self._values['Z0'] = [z]
        self._t = 0

        return self._obs(), {'Y': []}

    def action(self):
        vals = {**self._U, **self._values}

        if self._t == 0:
            return int(self.node_X0.sample(vals))

        return int(self.node_X1.sample(vals))

    def step(self, action):
        vals = {**self._U, **self._values}

        if self._t == 0:
            X0 = action
            vals['X0'] = [X0]
            self._values['X0'] = [X0]
            self._t = 1
            return self._obs(), 0.0, False, False, {'Y': []}

        X1 = action
        vals['X1'] = [X1]
        self._values['X1'] = [X1]

        y = self.node_y.sample(vals)
        self._values['Y'] = [y]

        self._t = 2
        return self._obs(), float(y), True, False, {'Y': [y]}

    @property
    def get_graph(self):
        return self.graph

In [7]:
class G2SCM(SCM):
    def __init__(self, graph, seed=None):
        super().__init__()
        self.rng = np.random.default_rng(seed)

        self.graph = graph
        self.node_names = [n['name'] for n in self.graph.nodes]
        self.index = {name: i for i, name in enumerate(self.node_names)}

        n = len(self.node_names)
        self._dir_adj = np.zeros((n, n), dtype=int)
        self._conf_adj = np.zeros((n, n), dtype=int)

        for e in self.graph.edges:
            i = self.index[e['from_']]
            j = self.index[e['to_']]

            if e['type_'] == 'directed':
                self._dir_adj[i, j] = 1
            elif e['type_'] == 'bidirected':
                self._conf_adj[i, j] = 1
                self._conf_adj[j, i] = 1

        self._conf_pairs = []
        seen = set()
        for a in self.node_names:
            for b in self.node_names:
                if a == b:
                    continue
                if self._conf_adj[self.index[a], self.index[b]] == 1:
                    key = tuple(sorted((a, b)))
                    if key not in seen:
                        seen.add(key)
                        self._conf_pairs.append(key)

        # stochastic bias used to generate many random instances
        self.U_bias = {pair: float(self.rng.uniform(-1.0, 1.0)) for pair in self._conf_pairs}

        # Z <--> X0
        z_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if 'Z0' in (a, b)]
        self.node_z = RandomBernoulliNode('Z0', parents=[], conf_inputs=z_conf, rng=self.rng)

        # Z <--> X0
        X0_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if set((a, b)) == set(('Z0', 'X0'))]
        self.node_X0 = RandomBernoulliNode('X0', parents=[], conf_inputs=X0_conf, rng=self.rng)

        # X0 -> X1
        X1_conf = []
        self.node_X1 = RandomBernoulliNode('X1', parents=['X0'], conf_inputs=X1_conf, rng=self.rng)

        # Z -> Y, X1 -> Y
        y_conf = []
        self.node_y = RandomBernoulliNode('Y', parents=['X1', 'Z0'], conf_inputs=y_conf, rng=self.rng)

        self.action_space = spaces.Discrete(2) # binary actions at each step
        self.observation_space = spaces.Dict({
            'Z': spaces.Sequence(spaces.Discrete(2)),
            'X': spaces.Sequence(spaces.Discrete(2))
        })

        self._t = 0
        self._values = {}
        self._U = {}

    def _sample_confounders(self):
        U = {}
        for a, b in self._conf_pairs:
            name = f'U_{a}_{b}'
            p = 1 / (1 + math.exp(-self.U_bias[(a, b)]))
            U[name] = 1 if self.rng.random() < p else 0
        return U

    def _obs(self):
        if self._t == 0:
            return {'Z': self._values['Z0'], 'X': []}
        elif self._t == 1:
            return {'Z': self._values['Z0'], 'X': [self._values['X0'][0]]}

        return {'Z': self._values['Z0'], 'X': [self._values['X0'][0], self._values['X1'][0]]}

    def reset(self, seed=None, options=None):
        if seed is not None:
            self.rng = np.random.default_rng(seed)

        self._values = {}
        self._U = self._sample_confounders()

        vals = {u: v for u, v in self._U.items()}
        z = self.node_z.sample(vals)
        self._values['Z0'] = [z]
        self._t = 0
        return self._obs(), {'Y': []}

    def action(self):
        vals = {**self._U, **self._values}
        if self._t == 0:
            return int(self.node_X0.sample(vals))
        return int(self.node_X1.sample(vals))

    def step(self, action):
        vals = {**self._U, **self._values}

        if self._t == 0:
            X0 = action
            vals['X0'] = [X0]
            self._values['X0'] = [X0]
            self._t = 1
            return self._obs(), 0.0, False, False, {'Y': []}

        X1 = action
        vals['X1'] = [X1]
        self._values['X1'] = [X1]

        y = self.node_y.sample(vals)
        self._values['Y'] = [y]

        self._t = 2
        return self._obs(), float(y), True, False, {'Y': [y]}

    @property
    def get_graph(self):
        return self.graph

In [8]:
class G3SCM(SCM):
    def __init__(self, graph, seed=None):
        super().__init__()
        self.rng = np.random.default_rng(seed)

        self.graph = graph
        self.node_names = [n['name'] for n in self.graph.nodes]
        self.index = {name: i for i, name in enumerate(self.node_names)}

        n = len(self.node_names)
        self._dir_adj = np.zeros((n, n), dtype=int)
        self._conf_adj = np.zeros((n, n), dtype=int)

        for e in self.graph.edges:
            i = self.index[e['from_']]
            j = self.index[e['to_']]
            if e['type_'] == 'directed':
                self._dir_adj[i, j] = 1
            elif e['type_'] == 'bidirected':
                self._conf_adj[i, j] = 1
                self._conf_adj[j, i] = 1

        self._conf_pairs = []
        seen = set()
        for a in self.node_names:
            for b in self.node_names:
                if a == b:
                    continue
                if self._conf_adj[self.index[a], self.index[b]] == 1:
                    key = tuple(sorted((a, b)))
                    if key not in seen:
                        seen.add(key)
                        self._conf_pairs.append(key)

        # stochastic bias used to generate many random instances
        self.U_bias = {pair: float(self.rng.uniform(-1.0, 1.0)) for pair in self._conf_pairs}

        # Z <--> X0
        z_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if 'Z0' in (a, b)]
        self.node_z = RandomBernoulliNode('Z0', parents=[], conf_inputs=z_conf, rng=self.rng)

        # Z <--> X0
        X0_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if set((a, b)) == set(('Z0', 'X0'))]
        self.node_X0 = RandomBernoulliNode('X0', parents=[], conf_inputs=X0_conf, rng=self.rng)

        # X0 -> X1
        X1_conf = []
        self.node_X1 = RandomBernoulliNode('X1', parents=['X0'], conf_inputs=X1_conf, rng=self.rng)

        # Z -> Y, X1 -> Y
        y_conf = []
        self.node_y = RandomBernoulliNode('Y', parents=['X1', 'Z0'], conf_inputs=y_conf, rng=self.rng)

        self.action_space = spaces.Discrete(2) # binary actions at each step
        self.observation_space = spaces.Dict({
            'Z': spaces.Sequence(spaces.Discrete(2)),
            'X': spaces.Sequence(spaces.Discrete(2))
        })

        self._t = 0
        self._values = {}
        self._U = {}

    def _sample_confounders(self):
        U = {}
        for a, b in self._conf_pairs:
            name = f'U_{a}_{b}'
            p = 1 / (1 + math.exp(-self.U_bias[(a, b)]))
            U[name] = 1 if self.rng.random() < p else 0
        return U

    def _obs(self):
        if self._t == 0:
            return {'Z': [], 'X': []}
        elif self._t == 1:
            return {'Z': self._values['Z0'], 'X': [self._values['X0'][0]]}

        return {'Z': self._values['Z0'], 'X': [self._values['X0'][0], self._values['X1'][0]]}

    def reset(self, seed=None, options=None):
        if seed is not None:
            self.rng = np.random.default_rng(seed)

        self._values = {}
        self._U = self._sample_confounders()
        self._t = 0
        return self._obs(), {'Y': []}

    def action(self):
        vals = {**self._U, **self._values}

        if self._t == 0:
            return int(self.node_X0.sample(vals))

        return int(self.node_X1.sample(vals))

    def step(self, action):
        vals = {**self._U, **self._values}

        if 'Z0' not in self._values or len(self._values['Z0']) == 0:
            z = self.node_z.sample(vals)
            self._values['Z0'] = [z]

        if self._t == 0:
            X0 = action
            vals['X0'] = [X0]
            self._values['X0'] = [X0]
            self._t = 1
            return self._obs(), 0.0, False, False, {'Y': []}

        X1 = action
        vals['X1'] = [X1]
        self._values['X1'] = [X1]

        y = self.node_y.sample(vals)
        self._values['Y'] = [y]

        self._t = 2
        return self._obs(), float(y), True, False, {'Y': [y]}

    @property
    def get_graph(self):
        return self.graph

In [9]:
class G4SCM(SCM):
    def __init__(self, graph, seed=None):
        super().__init__()
        self.rng = np.random.default_rng(seed)

        self.graph = graph
        self.node_names = [n['name'] for n in self.graph.nodes]
        self.index = {name: i for i, name in enumerate(self.node_names)}

        n = len(self.node_names)
        self._dir_adj = np.zeros((n, n), dtype=int)
        self._conf_adj = np.zeros((n, n), dtype=int)

        for e in self.graph.edges:
            i = self.index[e['from_']]
            j = self.index[e['to_']]
            if e['type_'] == 'directed':
                self._dir_adj[i, j] = 1
            elif e['type_'] == 'bidirected':
                self._conf_adj[i, j] = 1
                self._conf_adj[j, i] = 1

        self._conf_pairs = []
        seen = set()
        for a in self.node_names:
            for b in self.node_names:
                if a == b:
                    continue
                if self._conf_adj[self.index[a], self.index[b]] == 1:
                    key = tuple(sorted((a, b)))
                    if key not in seen:
                        seen.add(key)
                        self._conf_pairs.append(key)

        # stochastic bias used to generate many random instances
        self.U_bias = {pair: float(self.rng.uniform(-1.0, 1.0)) for pair in self._conf_pairs}

        # Z <--> X0, Z <--> Y, Z -> X1
        z_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if 'Z0' in (a, b)]
        self.node_z = RandomBernoulliNode('Z0', parents=[], conf_inputs=z_conf, rng=self.rng)

        # Z <--> X0
        X0_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if set((a, b)) == set(('Z0', 'X0'))]
        self.node_X0 = RandomBernoulliNode('X0', parents=[], conf_inputs=X0_conf, rng=self.rng)

        # Z -> X1
        X1_conf = []
        self.node_X1 = RandomBernoulliNode('X1', parents=['Z0'], conf_inputs=X1_conf, rng=self.rng)

        # X0 -> Y, X1 -> Y, Z <--> Y
        y_conf = [f'U_{a}_{b}' for (a, b) in self._conf_pairs if set((a, b)) == set(('Z0', 'Y'))]
        self.node_y = RandomBernoulliNode('Y', parents=['X0', 'X1'], conf_inputs=y_conf, rng=self.rng)

        self.action_space = spaces.Discrete(2) # binary actions at each step
        self.observation_space = spaces.Dict({
            'Z': spaces.Sequence(spaces.Discrete(2)),
            'X': spaces.Sequence(spaces.Discrete(2))
        })

        self._t = 0
        self._values = {}
        self._U = {}

    def _sample_confounders(self):
        U = {}
        for a, b in self._conf_pairs:
            name = f'U_{a}_{b}'
            p = 1 / (1 + math.exp(-self.U_bias[(a, b)]))
            U[name] = 1 if self.rng.random() < p else 0
        return U

    def _obs(self):
        if self._t == 0:
            return {'Z': [], 'X': []}
        elif self._t == 1:
            return {'Z': self._values['Z0'], 'X': [self._values['X0'][0]]}

        return {'Z': self._values['Z0'], 'X': [self._values['X0'][0], self._values['X1'][0]]}

    def reset(self, seed=None, options=None):
        if seed is not None:
            self.rng = np.random.default_rng(seed)

        self._values = {}
        self._U = self._sample_confounders()
        self._t = 0
        return self._obs(), {'Y': []}

    def action(self):
        vals = {**self._U, **self._values}
        if self._t == 0:
            return int(self.node_X0.sample(vals))
        return int(self.node_X1.sample(vals))

    def step(self, action):
        vals = {**self._U, **self._values}

        if 'Z0' not in self._values or len(self._values['Z0']) == 0:
            z = self.node_z.sample(vals)
            self._values['Z0'] = [z]

        if self._t == 0:
            X0 = action
            vals['X0'] = [X0]
            self._values['X0'] = [X0]
            self._t = 1
            return self._obs(), 0.0, False, False, {'Y': []}

        X1 = action
        vals['X1'] = [X1]
        self._values['X1'] = [X1]

        y = self.node_y.sample(vals)
        self._values['Y'] = [y]

        self._t = 2
        return self._obs(), float(y), True, False, {'Y': [y]}

    @property
    def get_graph(self):
        return self.graph

In [10]:
class GXPCH(PCH):
    def __init__(self, x, graph, seed=None):
        if x == 1:
            self.env = G1SCM(graph, seed=seed)
        elif x == 2:
            self.env = G2SCM(graph, seed=seed)
        elif x == 3:
            self.env = G3SCM(graph, seed=seed)
        elif x == 4:
            self.env = G4SCM(graph, seed=seed)

        super().__init__()
        self.last_obs = None

    @property
    def get_graph(self):
        return self.env.get_graph

    def reset(self, seed=None, options=None):
        obs, info = self.env.reset(seed=seed, options=options)
        self.last_obs = obs
        return obs, info

    def see(self, behavioral_policy=None, show_reward=False):
        if self.last_obs is None:
            self.last_obs, _ = self.reset()

        if behavioral_policy is None:
            action = self.env.action()
        else:
            action = behavioral_policy(dict(self.last_obs))

        obs, reward, terminated, truncated, info = self.env.step(action)
        self.last_obs = obs
        info['natural_action'] = action
        return obs, reward, terminated, truncated, info

    def do(self, do_policy, show_reward=False):
        if self.last_obs is None:
            self.last_obs, _ = self.env.reset()

        action = do_policy(self.last_obs)
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.last_obs = obs
        info['action'] = action
        return obs, reward, terminated, truncated, info

### BC Method Suite Setup

In [11]:
g1_env = GXPCH(1, G1, seed=seed)
g2_env = GXPCH(2, G2, seed=seed)
g3_env = GXPCH(3, G3, seed=seed)
g4_env = GXPCH(4, G4, seed=seed)

In [12]:
# Causal BC
# G1, G2, and G3 are imitable; G4 is not
g1_Z_sets = {'X0': set(), 'X1': set()}
g2_Z_sets = {'X0': {'Z0'}, 'X1': {'Z0'}}
g3_Z_sets = {'X0': set(), 'X1': {'Z0'}}

In [13]:
# BC - Observed Parents
g1_obs_parents_sets = {'X0': set(), 'X1': {'X0'}}
g2_obs_parents_sets = {'X0': set(), 'X1': {'X0'}}
g3_obs_parents_sets = {'X0': set(), 'X1': {'X0'}}
g4_obs_parents_sets = {'X0': set(), 'X1': {'Z0'}}

In [14]:
# BC - All Observed
g1_all_obs_sets = {'X0': {'Z0'}, 'X1': {'Z0', 'X0'}}
g2_all_obs_sets = {'X0': {'Z0'}, 'X1': {'Z0', 'X0'}}
g3_all_obs_sets = {'X0': set(), 'X1': {'Z0', 'X0'}}
g4_all_obs_sets = {'X0': set(), 'X1': {'Z0', 'X0'}}

### SCM Generation and BC Execution

In [15]:
def evaluate(env_type, graph_number, graph, Z_sets, num_scms=100, num_trajs=100, seed=0):
    gaps = []

    for k in range(num_scms):
        s = seed + k
        env = env_type(graph_number, graph, seed=s)

        # measure expert performance
        records = collect_expert_trajectories(env, num_episodes=num_trajs, max_steps=2, seed=s, show_progress=False)
        expert_EY = np.mean([r['reward'] for r in records if r['terminated']])

        policy = train_policies(env, records, Z_sets, max_epochs=100, seed=s)

        # measure imitator performance
        rollout = eval_policy(env, policy, num_episodes=num_trajs, seed=s)
        rewards = [ep['Y'][-1] for ep in rollout]
        imitator_EY = np.mean(rewards)

        gaps.append(abs(imitator_EY - expert_EY))

    return np.mean(gaps), np.std(gaps)

In [16]:
causal = {
    'G1': evaluate(GXPCH, 1, G1, g1_Z_sets),
    'G2': evaluate(GXPCH, 2, G2, g2_Z_sets),
    'G3': evaluate(GXPCH, 3, G3, g3_Z_sets)
}

obs_parents = {
    'G1': evaluate(GXPCH, 1, G1, g1_obs_parents_sets),
    'G2': evaluate(GXPCH, 2, G2, g2_obs_parents_sets),
    'G3': evaluate(GXPCH, 3, G3, g3_obs_parents_sets),
    'G4': evaluate(GXPCH, 4, G4, g4_obs_parents_sets)
}

all_obs = {
    'G1': evaluate(GXPCH, 1, G1, g1_all_obs_sets),
    'G2': evaluate(GXPCH, 2, G2, g2_all_obs_sets),
    'G3': evaluate(GXPCH, 3, G3, g3_all_obs_sets),
    'G4': evaluate(GXPCH, 4, G4, g4_all_obs_sets)
}

### Results

In [17]:
def format(gap):
    if gap == 'Not Imitable':
        return gap
    
    mean, std = gap
    return f'{mean:.2f} ± {std:.2f}'

table_data = []
for graph in ['G1', 'G2', 'G3', 'G4']:
    row = {
        'Graph': graph,
        'Causal BC': format(causal.get(graph, 'Not Imitable')),
        'Obs Parents': format(obs_parents.get(graph, 'Not Imitable')),
        'All Observed': format(all_obs.get(graph, 'Not Imitable'))
    }
    table_data.append(row)

df = pd.DataFrame(table_data)
df = df[['Graph', 'Causal BC', 'Obs Parents', 'All Observed']]
df

,Graph,Causal BC,Obs Parents,All Observed
0,G1,0.08 ± 0.05,0.07 ± 0.05,0.07 ± 0.05
1,G2,0.07 ± 0.06,0.07 ± 0.06,0.07 ± 0.06
2,G3,0.07 ± 0.06,0.07 ± 0.06,0.07 ± 0.06
3,G4,Not Imitable,0.07 ± 0.06,0.07 ± 0.06
